In [0]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.003

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# Bidirectional recurrent neural network (many-to-one)
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # RNN
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        #self.fc = nn.Linear(hidden_size, num_classes)

        # bidirectional RNN
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  # 2 for bidirection, 隐层包含向前层和向后层两层，所以隐层共有两倍的Hidden_size
    
    def forward(self, x):
        # Set initial states
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

model = BiRNN(input_size, hidden_size, num_layers, num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

0it [00:00, ?it/s]

9920512it [00:01, 9509015.05it/s]                            


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 96498.30it/s]            
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw


1654784it [00:00, 2308927.86it/s]                            
0it [00:00, ?it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw


8192it [00:00, 32066.98it/s]            


Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!
Epoch [1/2], Step [100/600], Loss: 0.7616
Epoch [1/2], Step [200/600], Loss: 0.4671
Epoch [1/2], Step [300/600], Loss: 0.1048
Epoch [1/2], Step [400/600], Loss: 0.1221
Epoch [1/2], Step [500/600], Loss: 0.0824
Epoch [1/2], Step [600/600], Loss: 0.1042
Epoch [2/2], Step [100/600], Loss: 0.0809
Epoch [2/2], Step [200/600], Loss: 0.1314
Epoch [2/2], Step [300/600], Loss: 0.0843
Epoch [2/2], Step [400/600], Loss: 0.0331
Epoch [2/2], Step [500/600], Loss: 0.1306
Epoch [2/2], Step [600/600], Loss: 0.1228
Test Accuracy of the model on the 10000 test images: 97.65 %


In [19]:
# Ref: https://zhuanlan.zhihu.com/p/39191116
import torch
import torch.nn as nn

bilstm = nn.LSTM(input_size=10, hidden_size=20, num_layers=2, bidirectional=True)

input = torch.randn(5, 3, 10)
print('\ninput shape: ', input.shape)
print("input:", input[:, :, 0])

h0 = torch.randn(4, 3, 20) # 4 = 2 * num_layers(2), batch size=3, hidden_size=20
print('\nh0 shape: ', h0.shape)
print(h0[:, :, 0])

c0 = torch.randn(4, 3, 20) # 4 = 2 * num_layers(2), batch size=3, hidden_size=20
print('\nc0 shape: ', c0.shape)

output, (hn, cn) = bilstm(input, (h0, c0))
print('\noutput shape: ', output.shape)
print('\nhn shape: ', hn.shape)
print('\ncn shape: ', cn.shape)

# print(output[4, 0, :20] == hn[2, 0])
# print(output[0, 0, 20:] == hn[3, 0])
print(hn[:, :, 0])


input shape:  torch.Size([5, 3, 10])
input: tensor([[-0.8767,  0.2394, -0.0512],
        [-0.1768, -1.5145, -0.0358],
        [-1.8995, -0.1540,  0.7092],
        [-0.1947,  0.3610, -2.2294],
        [ 0.2924,  0.8682, -0.3173]])

h0 shape:  torch.Size([4, 3, 20])
tensor([[ 0.3573, -0.7867,  0.6818],
        [-1.1077,  1.0407, -0.9948],
        [-2.1235,  1.8169,  0.2659],
        [-0.7552,  0.1324, -1.8419]])

c0 shape:  torch.Size([4, 3, 20])

output shape:  torch.Size([5, 3, 40])

hn shape:  torch.Size([4, 3, 20])

cn shape:  torch.Size([4, 3, 20])
tensor([[-0.1382, -0.1301, -0.1610],
        [ 0.0108, -0.0451,  0.0297],
        [-0.1500, -0.1486,  0.0622],
        [-0.1159, -0.1064, -0.0624]], grad_fn=<SelectBackward>)
